In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
data = pd.read_csv("data.csv")

In [3]:
data['Cuisines'] = data['Cuisines'].fillna('Unknown')
data['Rating text'] = data['Rating text'].fillna('Unknown')

In [4]:
boolean_columns = ['Has Table booking', 'Has Online delivery', 'Is delivering now', 'Switch to order menu']
for col in boolean_columns:
    data[col] = data[col].map({'Yes': 1, 'No': 0})

In [5]:
data['Features'] = (
    data['Cuisines'] + " " + data['Currency'] + " " + data['Rating text'] + " " + "OnlineDelivery_" + data['Has Online delivery'].astype(str) + " " + "TableBooking_" + data['Has Table booking'].astype(str) + " " + "PriceRange_" + data['Price range'].astype(str)
)

In [6]:
data['Features'] = data['Features'].str.replace(',', '').str.lower()
vectorizer = CountVectorizer()
feature_matrix = vectorizer.fit_transform(data['Features'])
cosine_sim = cosine_similarity(feature_matrix, feature_matrix)

In [ ]:
def recommend_restaurants(user_input, data, cosine_sim, top_n=5):

    user_features = vectorizer.transform([user_input.lower()])
    user_sim = cosine_similarity(user_features, feature_matrix)

    sim_scores = list(enumerate(user_sim[0]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    top_indices = [i[0] for i in sim_scores[:top_n]]
    recommendations = data.iloc[top_indices]

    return recommendations[['Restaurant Name', 'Cuisines', 'Currency', 'Aggregate rating', 'Price range', 
                             'Has Online delivery', 'Has Table booking']]

while True:
    print("\nEnter your preferences (e.g., 'North Indian Affordable OnlineDelivery_1'):")
    user_preference = input("Preferences: ")
    
    if user_preference.lower() in ['exit', 'quit']:
        print("Exiting recommendation system. Goodbye!")
        break

    print("\nRecommended Restaurants:")
    recommendations = recommend_restaurants(user_preference, data, cosine_sim, top_n=5)
    print(recommendations.to_string(index=False))

    print("\nWould you like to try again? Type 'exit' to quit.")


Enter your preferences (e.g., 'North Indian Affordable OnlineDelivery_1'):


Preferences:  chinese



Recommended Restaurants:
               Restaurant Name                                     Cuisines           Currency  Aggregate rating  Price range  Has Online delivery  Has Table booking
Kaveri Restaurant And Caterers North Indian, Chinese, South Indian, Chinese Indian Rupees(Rs.)               3.4            1                    0                  0
Kaveri Restaurant And Caterers North Indian, Chinese, South Indian, Chinese Indian Rupees(Rs.)               4.0            1                    0                  0
  House of China Restaurant II                                      Chinese          Dollar($)               3.8            1                    0                  0
                  Chang Garden                                      Chinese          Dollar($)               3.5            1                    0                  0
                  Hunan Palace                                      Chinese          Dollar($)               3.8            2                   